In [1]:
import polars as pl

In [2]:
df = pl.read_parquet("data/*.parquet")
df.head()

created_utc,title,upvotes,num_comments,upvote_ratio,is_created_from_ads_ui,total_awards_received,num_reports,image_url,is_video,sub_reddit,created_at
i64,str,i64,i64,f64,bool,i64,null,str,bool,str,datetime[μs]
1767362325,"""hees jhus Ate....""",984,6,0.99,false,0,null,"""https://i.redd.it/9qtpppaezxag…",false,"""dankmemes""",2026-01-02 13:58:45
1767356112,"""Apparently we can and apparent…",999,16,0.96,false,0,null,"""https://i.redd.it/3p1mf4v3hxag…",false,"""dankmemes""",2026-01-02 12:15:12
1767318516,"""Socializing in your 30s""",3624,33,0.98,false,0,null,"""https://i.redd.it/27g4jmfbduag…",false,"""dankmemes""",2026-01-02 01:48:36
1767320312,"""This year surely right?.......…",2392,132,0.97,false,0,null,"""https://i.redd.it/81zp4kq9iuag…",false,"""dankmemes""",2026-01-02 02:18:32
1767326064,"""Wish me luck, boys!""",1154,31,0.98,false,0,null,"""https://i.redd.it/bpz0m0erzuag…",false,"""dankmemes""",2026-01-02 03:54:24


In [3]:
df.shape

(1500, 12)

It would be wise to label enginner our variable here. And them check how well of a job we did, after we evaluate it with spearmasn correlation we can proceed

In [4]:

import time
now = time.time()

# Label enginnering
df = (
    df.with_columns(
        ((now - pl.col("created_utc")) / 3600)
        .clip(lower_bound=0)
        .alias("age_hours")
    )
    .with_columns(
        (pl.col("upvotes") + 2.0 * pl.col("num_comments"))
        .alias("engagement")
    )
    .with_columns(
        
        (pl.col("engagement") / (pl.col("age_hours") + 1) ** 0.5)
        .alias("engagement_score")
    )
)


In [5]:
# Sanity check lets see if these memes are firew
with pl.Config(fmt_str_lengths=50):
    print(df.sort(by="engagement_score",descending=True)["image_url"][0:5])

shape: (5,)
Series: 'image_url' [str]
[
	"https://i.redd.it/cu34eahcemag1.jpeg"
	"https://i.redd.it/45siaxe0rrag1.png"
	"https://i.redd.it/806kgphyjkag1.jpeg"
	"https://i.redd.it/phoavy4rxhag1.jpeg"
	"https://i.redd.it/2u7ppp4m1nag1.png"
]


In [ ]:
import cv2
import easyocr
import numpy as np

img = cv2.imread(r'data/test_2.jpeg')

img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

reader = easyocr.Reader(['en'])
result = reader.readtext("https://i.redd.it/cu34eahcemag1.jpeg",detail=0,paragraph=True)
result

Progress: |████████████████████████████████████████████████████| 105.8% Complete

[[[[336, 1], [691, 1], [691, 115], [336, 115]], 'FINALLYL'],
 [[[1, 777], [859, 777], [859, 1018], [1, 1018]],
  'THE GREAT MEME RESET OF 2026 imgflip com']]

In [7]:
cv2.imshow("processed", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.12.0) /io/opencv/modules/highgui/src/window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
cols = [
    "upvote_ratio",
    "upvotes",
    "num_comments",
    "total_awards_received",
    "engagement",
    "age_hours",
    "num_reports"
]

# Ideally our age hours variable here, should be negatively correlated. To value new memes more, which is essential for a ranker
df.select([
    pl.corr(col, "engagement_score", method="spearman").alias(col)
    for col in cols
])

upvote_ratio,upvotes,num_comments,total_awards_received,engagement,age_hours,num_reports
f64,f64,f64,f64,f64,f64,f64
0.531248,0.85955,0.662046,NaN,0.865845,-0.18904,NaN


In [ ]:
df.group_by("sub_reddit").agg(
    pl.mean("engagement_score").alias("mean_score"),
    pl.len().alias("n_posts"),
)


sub_reddit,mean_score,n_posts
str,f64,u32
"""memes""",141.26531,500
"""Memes_Of_The_Dank""",7.812128,500
"""dankmemes""",112.784072,500
